In [1]:
from neo4j import GraphDatabase
import pandas as pd
from openai import OpenAI
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import json
from collections import Counter
import mysql.connector
import os
import re
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# DB接続処理
# Neo4jに接続
uri = "bolt://localhost:7687"
user = "neo4j"
password = "abcd7890"

# ドライバを作成
driver = GraphDatabase.driver(uri, auth=(user, password))

In [3]:
# DB接続処理
# MySQLに接続
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="foodb"
)

In [4]:
# ChatGPTを使うのでAPIキーを設定
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gpt_client = OpenAI(
    api_key=OPENAI_API_KEY
)

In [5]:
# ChatGPTを使って意味のある文章を生成する
def get_gpt_result(prompt):
    response = gpt_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


In [6]:
# Word2Vecモデルを学習
#model = Word2Vec(sentences=all_flavors, vector_size=100, window=5, min_count=1, sg=1)
model = Word2Vec.load("../datas/fine_tuned_word2vec.model")

# フレーバーのベクトルを確認
green_vector = model.wv['green']
print(f'{model.vector_size}, {len(green_vector)}')

# similar to の結果をjsonで返す
def similar_to_json(word):
    try:    
        similar_words = model.wv.most_similar(positive=[word])
        similar_words_dict = {word: score for word, score in similar_words}
        json_data = json.dumps(similar_words_dict, indent=4)
        return json_data
    except KeyError:
        return json.dumps({})

# vectorを返す
def get_word_vector(word):
    try:    
        return model.wv[word]
    except KeyError:
        print("error key: ", word)
        return [0.0] * model.vector_size 
        


100, 100


In [7]:
# テキストフォーマットの指定
def format_text(text):
    if text is None:
        return ""
    text = text.replace('(', '_')
    text = text.replace(')', '_')
    text = text.replace(', ', ',')
    text = text.replace('.', '')
    text = text.replace(',', '_')
    text = text.replace('-', ' ')
    text = text.replace(' ', '_')
    text = text.replace('\n', '')
    text = re.sub(r'[()&]', '', text)
    text = text.lower()
    return text

In [8]:
#
# GroupとSubGroupを初期化
#
# 古いデータを削除してリセット
def initialize_groups(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (f:FoodGroup) DETACH DELETE f;")
    tx.run("MATCH (f:FoodSubGroup) DETACH DELETE f;")

# 新規レコードを作成
def create_groups(tx, node, group_name):
    if group_name is None or group_name == "":
        return
    
    vec_key = format_text(group_name)
    most_similar = similar_to_json(vec_key)
    word_vector = get_word_vector(vec_key)
            
    query = f"""
        MERGE (m:{node} {{
            id: $id,
            name: $name,
            most_similar_json: $most_similar_json,
            word_vector: $word_vector
        }})
        """
    
    tx.run(query,
        id=vec_key,
        name=group_name,
        most_similar_json=most_similar,
        word_vector=word_vector)
    
 # グループとサブグループの関連付け
def create_group_connections(tx, group_name, subgroup_name):
    tx.run("""
        MATCH (f:FoodGroup {id: $group_name})
        MATCH (s:FoodSubGroup {id: $subgroup_name})
        MERGE (f)-[:CONTAINS]->(s)
        """,
        group_name=format_text(group_name),
        subgroup_name=format_text(subgroup_name))

# インデックス追加
def append_group_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:FoodGroup) REQUIRE f.id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:FoodSubGroup) REQUIRE f.id IS UNIQUE")

# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(initialize_groups)
    
    query = "select food_group, food_subgroup from foods where food_group is not null group by food_group, food_subgroup"
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_groups, "FoodGroup", row["food_group"])
        session.execute_write(create_groups, "FoodSubGroup", row["food_subgroup"])
        session.execute_write(create_group_connections, row["food_group"], row["food_subgroup"])
    session.execute_write(append_group_index)

In [9]:
#
# Foodを初期化
#
# 古いデータを削除してリセット
def initialize_foods(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (f:Food) DETACH DELETE f;")
    tx.run("MATCH (f:FoodSubType) DETACH DELETE f;")

# 新規レコードを作成
def create_foods(tx, food_name, food_name_scientific):
    search_query = " ".join([food_name or "", food_name_scientific or ""])
    vec_key = format_text(food_name)
    most_similar = similar_to_json(vec_key)
    word_vector = get_word_vector(vec_key)
            
    query = f"""
        MERGE (m:Food {{
            id: $id,
            name: $name,
            search_query: $search_query,
            most_similar_json: $most_similar_json,
            word_vector: $word_vector
        }})
        """
    
    tx.run(query,
        id=vec_key,
        name=food_name,
        search_query=search_query,
        most_similar_json=most_similar,
        word_vector=word_vector)
    
# 新規レコードを作成
def create_food_sub_type(tx, food_name, food_sub_type_name):
    vec_key = format_text(food_name)
    most_similar = similar_to_json(vec_key)
    word_vector = get_word_vector(vec_key)
            
    query = f"""
        MERGE (m:FoodSubType {{
            id: $id,
            name: $name,
            most_similar_json: $most_similar_json,
            word_vector: $word_vector
        }})
        """
    
    tx.run(query,
        id=vec_key,
        name=food_name,
        most_similar_json=most_similar,
        word_vector=word_vector)
    
# FoodとFoodSubTypeの関連付け
def create_food_connections(tx, food_name, food_sub_type_name):
    tx.run("""
        MATCH (f:Food {id: $food_id})
        MATCH (s:FoodSubType {id: $food_sub_type_id})
        MERGE (f)-[:HAS_SUBTYPE]->(s)
        """,
        food_id=format_text(food_name),
        food_sub_type_id=format_text(food_sub_type_name))
    
# グループとフードの関連付け
def create_food_group_connections(tx, food_name, group_name, subgroup_name):
    tx.run("""
        MATCH (f:Food {id: $food_id})
        MATCH (g:FoodGroup {id: $group_id})
        MERGE (f)-[:HAS_GROUP]->(s)
        """,
        food_id=format_text(food_name),
        group_id=format_text(group_name))

    tx.run("""
        MATCH (f:Food {id: $food_id})
        MATCH (g:FooSubGroup {id: $sub_group_id})
        MERGE (f)-[:HAS_SUB_GROUP]->(s)
        """,
        food_id=format_text(food_name),
        sub_group_id=format_text(subgroup_name))
    
# インデックス追加
def append_foods_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:Food) REQUIRE m.id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:FoodSubType) REQUIRE m.id IS UNIQUE")
    # 新しいインデックスを作成
    tx.run("CREATE FULLTEXT INDEX index_text_search FOR (n:Food) ON EACH [n.search_query]")

# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(initialize_foods)
    
    query = "select foods.name as food_name, foods.name_scientific as food_name_scientific, foods.food_group, foods.food_subgroup, orig_food_common_name from foods left join contents on foods.id = contents.food_id group by foods.name,contents.orig_food_common_name"
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_foods, row["food_name"], row["food_name_scientific"])
        session.execute_write(create_food_sub_type, row["food_name"], row["orig_food_common_name"])
        session.execute_write(create_food_connections, row["food_group"], row["food_subgroup"])
        session.execute_write(create_food_group_connections, row["food_name"], row["food_subgroup"], row["food_subgroup"])
    
    session.execute_write(append_foods_index)

error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(clementine&tangerine)
error key:  mandarin_orange_(cl

BufferError: Existing exports of data: object cannot be re-sized

In [113]:
#
# Compoundを初期化
#
# 古いデータを削除してリセット
def initialize_compound(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (f:Compound) DETACH DELETE f;")

# 新規レコードを作成
def create_compound(tx, compound_name):
    if compound_name is None or compound_name == "":
        return
    
    vec_key = format_text(compound_name)
    #most_similar = similar_to_json(vec_key)
    #word_vector = get_word_vector(vec_key)

    query = f"""
        MERGE (m:Compound {{
            id: $id,
            name: $name
        }})
        """
    
    tx.run(query,
        id=vec_key,
        name=compound_name)

# インデックス追加
def append_compound_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:Compound) REQUIRE f.id IS UNIQUE")

# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(initialize_compound)
    
    query = "select orig_source_name from contents group by orig_source_name"
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_compound, row["orig_source_name"])
    
    session.execute_write(append_compound_index)

KeyboardInterrupt: 

In [ ]:
#
# Aroma初期化
#
# 古いデータを削除してリセット
def initialize_aroma(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (f:Aroma) DETACH DELETE f;")

# 新規レコードを作成
def create_aroma(tx, aroma_name):
    if aroma_name is None or aroma_name == "":
        return
    
    vec_key = format_text(aroma_name)
    most_similar = similar_to_json(vec_key)
    word_vector = get_word_vector(vec_key)

    query = f"""
        MERGE (m:Aroma {{
            id: $id,
            name: $name,
            most_similar_json: $most_similar_json,
            word_vector: $word_vector
        }})
        """
    
    tx.run(query,
        id=vec_key,
        name=aroma_name)

# インデックス追加
def append_aroma_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:Aroma) REQUIRE f.id IS UNIQUE")

# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(initialize_aroma)
    
    query = "select name from flavors group by name"
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_aroma, row["name"])
    
    session.execute_write(append_aroma_index)

In [115]:
#
# CompoundをFoodSubTypeと紐づける
# 
# mg/100gのデータを取得

In [ ]:
#
# AromaとCompoundを紐づける
# 